transformers necesarios para abordar el problema con Bert.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install --upgrade fsspec
!pip install fsspec==0.9.0

In [ ]:
!pip install transformers
!pip3 install datasets
!pip3 install sentencepiece

In [ ]:
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import GlobalAveragePooling1D, Dense
from tensorflow.keras import  Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

Inicialización de TPU

In [ ]:
print("Tensorflow version " + tf.__version__)
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

carga de archivos, entrenamiento y testing

In [ ]:
max_len = 120
batch_size = 128
test = pd.read_csv("../input/contradictory-my-dear-watson/test.csv")
train1 = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")

In [ ]:
test1 = test

analisis de los idiomas en el data set, para elegir modelo que los contenga

In [ ]:
train1['language'].value_counts(normalize=True)*100

según https://www.kaggle.com/yoohyuck/multilingual-roberta aumentar el dataset de entrenamiento muestra mejoras en el resultado, usa para ello el dataset de huggingface, replicaré esa consideración.

In [ ]:
from datasets import load_dataset

In [ ]:
def load_mnli(use_validation=True):
    result = []
    dataset = load_dataset('multi_nli')
    print(dataset['train'])
    keys = ['train', 'validation_matched','validation_mismatched'] if use_validation else ['train']
    for k in keys:
        for record in dataset[k]:
            c1, c2, c3 = record['premise'], record['hypothesis'], record['label']
            if c1 and c2 and c3 in {0,1,2}:
                result.append((c1,c2,c3,'en'))
    result = pd.DataFrame(result, columns=['premise','hypothesis', 'label','lang_abv'])
    return result

In [ ]:
mnli = load_mnli()

In [ ]:
total_train = train1[['id', 'premise', 'hypothesis','lang_abv', 'language', 'label']]
total_train

In [ ]:
mnli = mnli[['premise', 'hypothesis', 'lang_abv', 'label']]
mnli.insert(0, 'language', 'English')
mnli = mnli[['premise', 'hypothesis', 'lang_abv', 'language', 'label']]
mnli.insert(0, 'id', 'xxx')
mnli

In [ ]:
total_train = pd.concat([total_train, mnli], axis = 0)
total_train

In [ ]:
train1 = total_train[['premise', 'hypothesis', 'label']] #eliminamos los datos no relevantes, como el idioma o el id
train1

tokenizador con el modelo bert basico multilenguaje 

In [ ]:
modelo = 'joeddav/xlm-roberta-large-xnli'
tokenizer = AutoTokenizer.from_pretrained(modelo)

función para tokenizar palabras

In [ ]:
def token(x):
    tokens = list(tokenizer.tokenize(x))
    tokens.append('</s>')
    t = tokenizer.convert_tokens_to_ids(tokens)
    return t

función para codificar las premisas y las hipotesis

In [ ]:
def roberta_encode(hypotheses, premises, tokenizer):
  Pad = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
  sentence1 = tf.ragged.constant([token(s) for s in np.array(hypotheses)],  dtype=tf.int32)
  sentence2 = tf.ragged.constant([token(s) for s in np.array(premises)], dtype=tf.int32)
  cls = [tokenizer.convert_tokens_to_ids([tokenizer.cls_token])]*sentence1.shape[0]
  tokens = tf.concat([cls, sentence1, sentence2], axis=-1)
  tokens = tokens[:, :max_len] #quitar para la version full
  
  tokens = tokens.to_tensor(default_value=Pad)
  pad = max_len - tf.shape(tokens)[1]
  tokens = tf.pad(tokens, [[0, 0], [0, pad]], constant_values=Pad)
  input_word_ids = tf.reshape(tokens, [-1, max_len])

   
  input_mask = tf.cast(input_word_ids != Pad, tf.int32)
  input_mask = tf.reshape(input_mask, [-1, max_len])

  
  input_type_ids = tf.concat([tf.zeros_like(cls), tf.zeros_like(sentence1), tf.ones_like(sentence2)], 
                             axis=-1).to_tensor()


  inputs = {
    'input_word_ids': input_word_ids,
    'input_mask': input_mask,
    'input_type_ids': input_type_ids}

  return inputs

In [ ]:
def build_dataset(x, y, mode, batch_size):#función vista en varios notebook's
    if mode == "train":
        dataset = (
            tf.data.Dataset
            .from_tensor_slices((x, y))
            .repeat()
            .shuffle(5678)
            .batch(batch_size)
            .prefetch(tf.data.experimental.AUTOTUNE)
        )
    elif mode == "valid":
        dataset = (
            tf.data.Dataset
            .from_tensor_slices((x, y))
            .batch(batch_size)
            .cache()
            .prefetch(tf.data.experimental.AUTOTUNE)
        )
    elif mode == "test":
        dataset = (
            tf.data.Dataset
            .from_tensor_slices(x)
            .batch(batch_size)
            )
    else:
        raise NotImplementedError
    return dataset

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(train1[['premise', 'hypothesis']].values.tolist(), train1['label'], test_size=0.25, random_state=0)

In [ ]:
o = []
p = []
q = []
r = []
for i in range(len(x_train)):
  o.append(x_train[i][0])
  p.append(x_train[i][1])
for i in range(len(x_valid)):
  q.append(x_valid[i][0]) #cambio
  r.append(x_valid[i][1])

In [ ]:
x_train_ = roberta_encode(o, p, tokenizer)
x_valid_ = roberta_encode(q, r, tokenizer)

In [ ]:
train_dataset = build_dataset(x_train_, y_train, "train", batch_size)
valid_dataset = build_dataset(x_valid_, y_valid, "valid", batch_size)

In [ ]:
def build_model(model, max_len):
    
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    
    with strategy.scope():
        
        input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
        model = TFAutoModel.from_pretrained(modelo)
        roberta = model([input_word_ids])[0]
        output = GlobalAveragePooling1D()(roberta)
        output = Dense(3, activation='softmax')(output)
        
        model = Model(inputs=[input_word_ids], outputs = output)
        model.compile(optimizer=Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    model.summary()
    
    return model

In [ ]:
model = build_model(modelo, max_len)

In [ ]:
steps_per_epoch = len(x_train) // batch_size
stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', verbose=1, patience=2, mode='min', restore_best_weights=True)

In [ ]:
model.fit(train_dataset, validation_data=valid_dataset, steps_per_epoch=steps_per_epoch, epochs=4, callbacks=[stop])

hacemos la predicción

In [ ]:
test

In [ ]:
test = test[['premise', 'hypothesis']] #eliminamos los datos no relevantes, como el idioma o el id
test

In [ ]:
test_data = roberta_encode(test.premise.values, test.hypothesis.values, tokenizer)
test_data

In [ ]:
test_data  = build_dataset(test_data, None, "test", batch_size)
test_data

In [ ]:
predictions = [np.argmax(i) for i in model.predict(test_data)]


creamos y guardamos el archvo a enviar

In [ ]:
submission = test1.id.copy().to_frame()
print(submission)

In [ ]:
submission['prediction'] = predictions

In [ ]:
submission.head()

In [ ]:
submission.reset_index(drop=True, inplace=True)

In [ ]:
print(submission)

In [ ]:
submission.to_csv("submission.csv", index = False)